In [1]:
import os
import pandas as pd
from glob import glob
from pathlib import Path

In [3]:
# Define paths
label_path = Path('dataset/training/label_2')
output_path = Path('labels_yolo')
output_path.mkdir(parents=True, exist_ok=True)  # Create output directory if it doesn't exist

In [5]:
# Define label columns
label_cols = [
    'label', 'truncated', 'occluded', 'alpha',
    'bbox_xmin', 'bbox_ymin', 'bbox_xmax',
    'bbox_ymax', 'dim_height', 'dim_width', 'dim_length',
    'loc_x', 'loc_y', 'loc_z', 'rotation_y', 'score'
]
columns_to_keep = ['label', 'bbox_xmin', 'bbox_ymin', 'bbox_xmax', 'bbox_ymax']

In [7]:
# Function to eliminate unnecessary variables
def eliminate_variables(label_file):
    data = pd.read_csv(
        label_file,
        sep=" ",
        names=label_cols[:15],
        usecols=columns_to_keep
    )
    return data

In [11]:
# Function to convert bounding boxes to YOLO format
def convert_bbox_to_yolo_format(bbox, img_width, img_height):
    x_center = ((bbox['bbox_xmin'] + bbox['bbox_xmax']) / 2) / img_width
    y_center = ((bbox['bbox_ymin'] + bbox['bbox_ymax']) / 2) / img_height
    width = (bbox['bbox_xmax'] - bbox['bbox_xmin']) / img_width
    height = (bbox['bbox_ymax'] - bbox['bbox_ymin']) / img_height
    return x_center, y_center, width, height

In [13]:
# Process the entire dataset
def process_dataset(label_path, img_width, img_height):
    for label_file in label_path.glob('*.txt'):
        data = eliminate_variables(label_file)
        data = data[data['label'] != 'DontCare']  # Ignore 'DontCare' labels
        data['label'] = data['label'].map(label_mapping)
        
        yolo_labels = []
        for _, row in data.iterrows():
            x_center, y_center, width, height = convert_bbox_to_yolo_format(row, img_width, img_height)
            yolo_labels.append(f"{int(row['label'])} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # Write to YOLO formatted text file
        output_file = output_path / label_file.name
        with open(output_file, 'w') as f:
            f.write("\n".join(yolo_labels))

In [15]:
# Define mapping for labels to categorical values
label_mapping = {
    'Car': 0, 'Van': 1, 'Truck': 2, 'Pedestrian': 3, 
    'Person_sitting': 4, 'Cyclist': 5, 'Tram': 6, 'Misc': 7, 'DontCare': 8
}

In [17]:
# Function to transform labels into categorical variables
def transform_to_categorical(data):
    data['label'] = data['label'].map(label_mapping)
    return data

In [21]:
# # Example usage
# img_width, img_height = 1242, 375  # Replace with the actual image width and height
# process_dataset(label_path, img_width, img_height)